In [1]:
# Python3
import sqlite3
from urllib import request, error
from sqlite3 import Error
from bs4 import BeautifulSoup, Tag

database = "./ACCT/database2.db"

headTblFields = """accNum text NOT NULL,
headId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
documentType text NOT NULL,
publicDocCount integer DEFAULT 0,
periodOfReport text,
filedDate text,
changedDate text,
schemaVersion text,
dateOfOriginalSubmission text,
notSubjectToSection16 integer DEFAULT 0,
issuerName text,
issuerCik text,
issuerIndustrialClassification text,
issuerIrs text,
issuerIncorpState text,
issuerFiscalYrEnd text,
issuerBusinessStreet1 text,
issuerBusinessCity text,
issuerBusinessState text,
issuerBusinessZip text,
issuerBusinessPhone text,
issuerMailStreet1 text,
issuerMailStreet2 text,
issuerMailCity text,
issuerMailState text,
issuerMailZip text,
issuerTradingSymbol text,
rptOwnerName text,
rptOwnerCik text DEFAULT 0,
rptOwnerFormType text,
rptOwnerSecAct text,
rptOwnerSecFileNum text,
rptOwnerFilmNum text,
rptOwnerBusinessPhone text,
rptOwnerStreet1 text,
rptOwnerStreet2 text,
rptOwnerCity text,
rptOwnerState text,
rptOwnerZipCode text,
rptOwnerStateDescription text,
rptOwnerMailStreet1 text,
rptOwnerMailStreet2 text,
rptOwnerMailCity text,
rptOwnerMailState text,
rptOwnerMailZip text,
rptOwnerisDirector integer DEFAULT 0,
rptOwnerisOfficer integer DEFAULT 0,
rptOwnerisTenPercentOwner integer DEFAULT 0,
rptOwnerisOther integer DEFAULT 0"""

dTTblFields = """accNum text NOT NULL,
dTId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
securityTitle text,
conversionOrExercisePrice text,
transactionDate text,
transactionFormType text,
transactionCode text,
equitySwapInvolved text,
transactionShares text,
transactionPricePerShare text,
transactionAcquiredDisposedCode text,
exerciseDate text,
expirationDate text,
underlyingSecurityTitle text,
underlyingSecurityShares text,
sharesOwnedFollowingTransaction text,
directOrIndirectOwnership text,
type text,
footNoteId text,"""

nDTTblFields = """accNum text NOT NULL,
nDTId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
securityTitle text,
transactionDate text,
transactionFormType text,
transactionCode text,
equitySwapInvolved text,
transactionTimelines text,
transactionShares text,
transactionPricePerShare text,
transactionAcquiredDisposedCode text,
sharesOwnedFollowingTransaction text,
directOrIndirectOwnership text,
natureOfOwnership text,
type text,
footNoteId text,"""

footNoteTblFields = """accNum text NOT NULL,
rowNumber integer DEFAULT 1,
footNoteId text PRIMARY KEY NOT NULL,
fId text,
originalTableType text,
footNoteField text,
footNote text"""

dTTblSql = dTTblFields + "FOREIGN KEY(footNoteId) REFERENCES form4footNote(footNoteId)"
nDTTblSql = nDTTblFields + "FOREIGN KEY(footNoteId) REFERENCES form4footNote(footNoteId)"

In [2]:
textHeadMap = {"ACCESSION NUMBER": "accNum",
              "CONFORMED SUBMISSION TYPE": "documentType",
              "PUBLIC DOCUMENT COUNT": "publicDocCount",
              "CONFORMED PERIOD OF REPORT": "periodOfReport",
              "FILED AS OF DATE": "filedDate",
              "DATE AS OF CHANGE": "changedDate"}
issuerMap = {"COMPANY CONFORMED NAME": "issuerName",
              "CENTRAL INDEX KEY": "issuerCik",
              "STANDARD INDUSTRIAL CLASSIFICATION": "issuerIndustrialClassification",
              "IRS NUMBER": "issuerIrs",
              "STATE OF INCORPORATION": "issuerIncorpState",
              "FISCAL YEAR END": "issuerFiscalYrEnd"}
issuerBizMap = {"STREET 1": "issuerBusinessStreet1",
              "CITY": "issuerBusinessCity",
              "STATE": "issuerBusinessState",
              "ZIP": "issuerBusinessZip",
              "BUSINESS PHONE": "issuerBusinessPhone"}
issuerMailMap = {"STREET 1": "issuerMailStreet1",
              "STREET 2": "issuerMailStreet2",
              "CITY": "issuerMailCity",
              "STATE": "issuerMailState",
              "ZIP": "issuerMailZip"}
ownerMap = {"COMPANY CONFORMED NAME": "rptOwnerName",
          "CENTRAL INDEX KEY": "rptOwnerCik",
          "FORM TYPE": "rptOwnerFormType",
          "SEC ACT": "rptOwnerSecAct",
          "SEC FILE NUMBER": "rptOwnerSecFileNum",
          "FILM NUMBER": "rptOwnerFilmNum",
          "STREET 1": "rptOwnerMailStreet1",
          "STREET 2": "rptOwnerMailStreet2",
          "CITY": "rptOwnerMailCity",
          "STATE": "rptOwnerMailState",
          "ZIP": "rptOwnerMailZip",
          "BUSINESS PHONE": "rptOwnerBusinessPhone"}
xml2SqlOwnFields = {"isDirector" : "rptOwnerisDirector",
                 "isOfficer": "rptOwnerisOfficer", 
                 "isTenPercentOwner": "rptOwnerisTenPercentOwner", 
                 "isOther": "rptOwnerisOther"}
xmlEtcHeaders = ["schemaVersion", "documentType", "periodOfReport", "dateOfOriginalSubmission", "notSubjectToSection16"]
xmlIssHeaders = ["issuerCik", "issuerName", "issuerTradingSymbol"]
xmlOwnHeaders = ["rptOwnerCik", "rptOwnerName", "rptOwnerStreet1", "rptOwnerStreet2", "rptOwnerCity", "rptOwnerState","rptOwnerZipCode", "rptOwnerStateDescription", "isDirector", "isOfficer", "isTenPercentOwner", "isOther"]

In [3]:
def parseSecHeader(lines, sqlMap, sqlDic):
    lines = lines.replace(":", "").split('\n')
    for line in lines:
        if "\t" in line:
            parts = line.split('\t')
            parts = [x for x in parts if x != '']
            if parts and parts[0] in sqlMap:
                sqlDic[sqlMap[parts[0]]] = parts[1]
    return sqlDic

def addXmlHeader(sqlDic, xmlFields, xml2SqlMap):
    for field in xmlFields:
        val = soup.find(field)
        if val:
            val = val.getText()
        key = field
        if xml2SqlMap and field in xml2SqlMap:
            key = xml2SqlMap[field]
        if key == 'dateOfOriginalSubmission' and val:
            val = val.replace("-", "")
        sqlDic[key] = val
        
def addXmlHeaderMultiple(sqlDics, xmlFields, xml2SqlMap, index):
    for field in xmlFields:
        key = field
        if xml2SqlMap and field in xml2SqlMap:
            key = xml2SqlMap[field]
        
        vals = soup.find_all(field)
        if vals:
            sqlDics[key] = vals[index].getText()
#         for i in range(len(sqlDics)):
#             sqlDics[i][key] = vals[i].getText()

def parseXml(xmlFieldList, transactionsList, transactionType, accNum):
    xmlLis = xmlFieldList.split('\n')
    xmlFields = []
    for field in xmlLis:
        fieldParts = field.split(' ')
        xmlFields.append(fieldParts[0])

    sqlDic = {}
    if transactionType == 'derivativeTransaction'or transactionType == 'derivativeHolding':
        footNoteTag = 'dt'
    else:
        footNoteTag = 'ndt'
    if transactionType == 'derivativeTransaction'or transactionType == 'nonDerivativeTransaction':
        entryType = 'transaction'
    else:
        entryType = 'holding'
    for row in range(len(transactionsList)):
        sqlDic[row+1] = {}
        sqlDic[row+1]['footnotes'] = []
        sqlDic[row+1]['type'] = entryType
        for field in xmlFields:
            if field == 'accNum':
                val = accNum
            elif field == 'rowNumber':
                val = row+1
            elif field == 'dTId' or field == 'nDTId':
                val = accNum+'-'+str(row+1)
                if entryType == 'holding':
                    val += '-h'
                print(val)
            elif field == 'footNoteId':
                continue
            elif field == 'transactionTimelines':
                field = 'transactionTimeliness'
            else:
                results = transactionsList[row].find(field)
                if results:
                    val = ''.join([str(x) for x in results.contents if x != '\n'])
                    for x in results.contents:
                        if x and isinstance(x, Tag):
                            fId = x.get('id')
                            if fId is not None:
                                footNoteId = accNum+'-'+ str(row+1)+'-'+footNoteTag + '-' + fId + '-' + field
                                if entryType == 'holding':
                                    footNoteId += '-h'
                                foot = {'accNum': accNum,
                                       'rowNumber': row+1,
                                       'footNoteId': footNoteId,
                                       'fId': fId,
                                       'originalTableType': transactionType,
                                       'footNoteField': field
                                       }
                                sqlDic[row+1]['footnotes'].append(foot)
                else:
                    val = None
            if field == 'transactionTimeliness':
                field = 'transactionTimelines'
            sqlDic[row+1][field] = val
    return sqlDic

def fillFootNoteText(footnotes):
    for note in footnotes:
        fId = note['fId']
        footnote = soup.find('footnote', {'id': fId})
        if footnote:
            note['footNote'] = footnote.contents[0].replace('\n', '')

In [4]:
### Database utility functions ###
def connectToDb(db):
    try:
        conn = sqlite3.connect(db)
        return conn
    except Error as e:
        print(e)

def createTable(conn, tableName, tableFields):
    try:
        c = conn.cursor()
        create_sql = "CREATE TABLE IF NOT EXISTS "+tableName+" (" + tableFields +");"
        c.execute(create_sql)
    except Error as e:
        print(e)
        
def insertToTable(table, dictionary, conn):
    columns = ', '.join(dictionary.keys())
    placeholders = ', '.join('?' * len(dictionary))
    sql = 'INSERT INTO ' + table + ' ({}) VALUES ({})'.format(columns, placeholders)
    try:
        conn.execute(sql, list(dictionary.values()))
    except Exception as e:
        print(e)
        conn.close()
        return
    conn.commit()
    
def insertToTransacTables(sql, tableName):
    for rowNum in sql.keys():
        row = sql[rowNum]
        footnotes = row.pop('footnotes')
        fillFootNoteText(footnotes)

        conn = connectToDb(database)
        insertToTable(tableName, row, conn)

        for note in footnotes:
            insertToTable('form4footNote', note, conn)


In [27]:
def parseHead(soup, accNum):
    sqlDic = {} 

    # SEC-HEADER
    sec_header = soup.find("ACCEPTANCE-DATETIME").getText()
    issuerBegin = sec_header.find("ISSUER")
    ownerBegin = sec_header.find("REPORTING-OWNER")

    # Process top of SEC-HEADER
    etcHeader = sec_header[:issuerBegin]
    sqlDic = parseSecHeader(etcHeader, textHeadMap, sqlDic)
    addXmlHeader(sqlDic, xmlEtcHeaders, None)

    # SEC-HEADER Issuer section
    issuerHead = sec_header[issuerBegin:ownerBegin]

    # Issuer company data
    sqlDic = parseSecHeader(issuerHead, issuerMap, sqlDic)
    addXmlHeader(sqlDic, xmlIssHeaders, None)

    bizBegin = issuerHead.find("BUSINESS ADDRESS")
    mailBegin = issuerHead.find("MAIL ADDRESS")

    # SEC-HEADER Issuer business address
    issuerHeadBiz = issuerHead[bizBegin:mailBegin]
    sqlDic = parseSecHeader(issuerHeadBiz, issuerBizMap, sqlDic)

    # SEC-HEADER Issuer mail address
    issuerHeadMail = issuerHead[mailBegin:]
    sqlDic = parseSecHeader(issuerHeadMail, issuerMailMap, sqlDic)

    # SEC-HEADER Reporting owners section & insert into table
    rOwners = sec_header.split("REPORTING-OWNER:")[1:]
    for index in range(len(rOwners)):
        sqlDic = parseSecHeader(rOwners[index], ownerMap, sqlDic)
        addXmlHeaderMultiple(sqlDic, xmlOwnHeaders, xml2SqlOwnFields, index)
        sqlDic['rowNumber'] = index+1 
        sqlDic['accNum'] = accNum
        sqlDic['headId'] = accNum + "-" + str(index+1)
        #print(sqlDic)
        #insertToTable('form4head', sqlDic, conn)

def parseTransacs(soup, accNum):
    # Process derivative transactions
    derivativeTransactions = soup.find_all("derivativeTransaction")
    sql = parseXml(dTTblFields, derivativeTransactions, 'derivativeTransaction', accNum)
#     print(sql)
    insertToTransacTables(sql, 'form4dT')

    derivativeHoldings = soup.find_all("derivativeHolding")
    sql = parseXml(dTTblFields, derivativeHoldings, 'derivativeHolding', accNum)
#     print(sql)
    insertToTransacTables(sql, 'form4dT')

    # Process non-derivative transactions
    nonDerivativeTransactions = soup.find_all("nonDerivativeTransaction")
    sqlNT = parseXml(nDTTblFields, nonDerivativeTransactions, 'nonDerivativeTransaction', accNum)
#     print(sqlNT)
    insertToTransacTables(sqlNT, 'form4nDT')

    nonDerivativeHoldings = soup.find_all("nonDerivativeHolding")
#     print('ndh', len(nonDerivativeHoldings))
    sqlNT = parseXml(nDTTblFields, nonDerivativeHoldings, 'nonDerivativeHolding', accNum)
#     print(sqlNT)
    insertToTransacTables(sqlNT, 'form4nDT')

In [25]:
conn = connectToDb(database)
createTable(conn, 'form4Head', headTblFields)
createTable(conn, 'form4dT', dTTblSql)
createTable(conn, 'form4nDT', nDTTblSql)
createTable(conn, 'form4footNote', footNoteTblFields)

#fname = "2007_4A_accNum" #fname = "flowers_com_inc2014.txt"
fname = "2017Form4.csv"
#fname = "2017Form4a" #400 4/a

with open(fname) as f:
    urls = f.read().splitlines()

#thisTime = urls #400 forms 4as
thisTime = urls[6050:8001] #5000 form 4s
#thisTime = urls[4001:4002]
for link in thisTime:
    links = link.split(",")
    url = "https://www.sec.gov/Archives/" + links[len(links)-1] #for 4s
    #url = "https://www.sec.gov/Archives/edgar/data/1305168/0000899243-17-004732.txt"
    #url = "https://www.sec.gov/Archives/" + link + ".txt" #for 2014form4a    
    try:
        response = request.urlopen(url)
    except error.HTTPError as err:
        failed.append(url)
        continue
    
    the_page = response.read()
    content = the_page.decode(encoding='latin-1')
    file = open("test", "w")
    file.write(content) 

    #parts = link.split("/")
    parts = links[len(links)-1].split('/')
    if len(parts) == 5: #.../data/1084869/000108486914000025/0001084869-14-000025.txt
        accNum = parts[len(parts)-3] + '/' +parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]
    if len(parts) == 4: #.../data/1214101/0001104659-07-084171.txt
        accNum = parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]
        
    # start parsing
    begin = content.find("<SEC-DOCUMENT>")
    end = content.find("-----END")
    xmlFile = content[begin:end]
    soup = BeautifulSoup(xmlFile, 'xml')
    #parseHead(soup, accNum)
    parseTransacs(soup, accNum)


1670541/0001127602-17-011526-1
1670541/0001127602-17-011526-2
1691425/0001179110-17-003774-1
1400395/0001127602-17-001099-1
1400395/0001209191-17-001789-1
1400395/0001209191-17-016371-1
1400395/0001209191-17-016371-1-h
1570353/0001437749-17-003818-1
1346101/0001279569-17-000011-1
1346101/0001279569-17-000011-1
1346101/0001279569-17-000011-2
1431535/0001228924-17-000001-1
1431535/0001228924-17-000001-2
1431535/0001228924-17-000001-1
1431535/0001228924-17-000001-2
1431535/0001228924-17-000001-3
1431535/0001228924-17-000001-4
1685904/0001127430-17-000009-1
1649101/0001551182-17-000045-1
1649101/0001551182-17-000045-2
1649101/0001551182-17-000069-1
1649101/0001551182-17-000069-1
1649101/0001551182-17-000069-2
1649101/0001551182-17-000069-1-h
1649101/0001551182-17-000081-1
1649101/0001551182-17-000081-2
1649101/0001551182-17-000081-1
1649101/0001551182-17-000081-2
1649101/0001551182-17-000081-3
1649101/0001551182-17-000081-4
1649101/0001551182-17-000081-1-h
1673985/0000950117-17-000157-1
16

1422142/0000899243-17-008099-1
1422142/0000899243-17-008150-1
1422142/0000899243-17-008151-1
1422142/0000899243-17-008156-1
1422142/0000899243-17-008159-1
1422142/0000899243-17-008160-1
1422142/0000899243-17-008161-1
1422142/0000899243-17-008162-1
1138776/0001192482-17-000046-1
1138776/0001192482-17-000047-1
1138776/0001192482-17-000048-1
1138776/0001686266-17-000002-1
1138776/0001686266-17-000002-2
1138776/0001686266-17-000002-3
1138776/0001686266-17-000002-1-h
1664569/0001209191-17-009529-1
1493978/0001577095-17-000037-1
1493978/0001577095-17-000056-1
1404624/0001140361-17-002699-1
1404624/0001104659-17-011343-1
1404624/0001104659-17-011343-2
1499268/0001104659-17-005808-1
1499268/0001104659-17-005809-1
1499268/0001104659-17-005809-1
1499268/0001104659-17-005810-1
1499268/0001104659-17-005811-1
1499268/0001104659-17-005811-1
1499268/0001104659-17-005812-1
1499268/0001104659-17-005812-2
1499268/0001104659-17-005812-3
1499268/0001104659-17-005812-1
1499268/0001104659-17-005813-1
149926

1362988/0001209191-17-003489-1
1362988/0001209191-17-003489-1-h
1362988/0001209191-17-009989-1
1362988/0001209191-17-009994-1
1362988/0001209191-17-009998-1
1362988/0001209191-17-010009-1
1362988/0001209191-17-010019-1
1362988/0001209191-17-010025-1
1362988/0001209191-17-010025-1-h
1607521/0000721748-17-000163-1
1607521/0000721748-17-000163-1-h
1532635/0000950142-17-000108-1
1532634/0000950142-17-000108-1
1638341/0001209191-17-020030-1
1638341/0001209191-17-020030-2
1638341/0001209191-17-020030-1-h
1293424/0001600781-17-000010-1
1293424/0001600781-17-000010-1-h
1454810/0000354707-17-000005-1
1454810/0000354707-17-000005-2
1454810/0000354707-17-000014-1
1557533/0001140361-17-009672-1
1557533/0001140361-17-010942-1
1557533/0001140361-17-011081-1
1557533/0001140361-17-012655-1
1557533/0001140361-17-013317-1
1557533/0001140361-17-013317-1
1557533/0001140361-17-014215-1
1607717/0001209191-17-018282-1
1607717/0001209191-17-018282-2
1607717/0001209191-17-020273-1
1517022/0001562180-17-000080-

1525976/0000947871-17-000207-1
1525976/0000947871-17-000207-2
1323202/0001209191-17-013617-1
1323202/0001209191-17-013617-2
1676146/0001609711-17-000056-1
1676146/0001609711-17-000056-2
1676146/0001609711-17-000056-3
1676146/0001609711-17-000056-4
1676146/0001609711-17-000056-5
1676146/0001609711-17-000056-1
1676146/0001609711-17-000056-2
1687664/0001140361-17-004967-1
1548336/0001127602-17-007747-1
1548336/0001127602-17-007747-1
1548336/0001127602-17-007747-2
1548336/0001127602-17-007747-3
1548336/0001127602-17-007747-4
1537972/0001127602-17-003112-1
1537972/0001127602-17-003112-1-h
1537972/0001127602-17-008633-1
1537972/0001127602-17-008633-2
1537972/0001127602-17-010164-1
1537972/0001127602-17-010164-1-h
1537972/0001127602-17-011176-1
1537972/0001127602-17-011176-1-h
1537972/0001127602-17-012190-1
1537972/0001127602-17-012190-1-h
1537972/0001127602-17-013379-1
1537972/0001127602-17-013379-2
1537972/0001127602-17-013379-1
1688865/0001209191-17-013738-1
1545654/0001225208-17-001107-1


1520262/0001127602-17-003730-1
1520262/0001127602-17-003730-1
1520262/0001127602-17-003730-2
1520262/0001127602-17-006681-1
1520262/0001127602-17-006681-1
1520262/0001127602-17-006681-2
1520262/0001127602-17-006681-3
1520262/0001127602-17-006681-4
1520262/0001127602-17-007439-1
1520262/0001127602-17-007439-2
1520262/0001127602-17-007440-1
1520262/0001127602-17-007440-2
1520262/0001127602-17-007447-1
1520262/0001127602-17-007447-2
1520262/0001127602-17-007473-1
1520262/0001127602-17-007473-2
1520262/0001127602-17-007474-1
1520262/0001127602-17-007474-2
1520262/0001127602-17-007477-1
1520262/0001127602-17-007477-2
1520262/0001127602-17-007479-1
1520262/0001127602-17-007479-2
1520262/0001127602-17-007482-1
1520262/0001127602-17-007482-2
1520262/0001127602-17-007489-1
1520262/0001127602-17-007489-2
1520262/0001127602-17-007489-1
1520262/0001127602-17-008675-1
1520262/0001127602-17-008675-1
1520262/0001127602-17-008675-2
1520262/0001127602-17-008675-1-h
1520262/0001127602-17-008675-2-h
1520

...

1568933/0001209191-17-017862-1
1568933/0001209191-17-017862-2
1568933/0001209191-17-020591-1
1568933/0001209191-17-020591-1
1568933/0001209191-17-020591-2
1362098/0001571049-17-002352-1
1362098/0001571049-17-002352-2
1362098/0001571049-17-002352-3
1362098/0001571049-17-002352-4
1362098/0001571049-17-002352-1
1362098/0001571049-17-002352-2
1362098/0001571049-17-002352-1-h
1505081/0001510192-17-000007-1
1505081/0001510192-17-000019-1
1505081/0001510192-17-000019-2
1505081/0001510192-17-000019-3
1505081/0001510192-17-000019-4
1506097/0001209191-17-011873-1
1506097/0001209191-17-011873-2
1506097/0001209191-17-011873-3
1506097/0001209191-17-011873-4
1506097/0001209191-17-011873-1
1506097/0001209191-17-015389-1
1529284/0001179110-17-000429-1
1529284/0001179110-17-000429-1
1529284/0001179110-17-000429-2
1529284/0001179110-17-000429-3
1529284/0001179110-17-000429-4
1529284/0001179110-17-000429-5
1529284/0001179110-17-000429-6
1529284/0001179110-17-000429-7
1529284/0001179110-17-000429-8
152928

1309403/0001209191-17-021352-1
1309403/0001209191-17-023515-1
1309403/0001209191-17-023515-2
1309403/0001209191-17-023515-3
1309403/0001209191-17-023515-1
1309403/0001209191-17-023515-2
1353876/0001127602-17-008370-1-h
1353876/0001127602-17-008370-2-h
1353876/0001127602-17-008370-1
1353876/0001127602-17-008370-1-h
1353876/0001127602-17-009249-1
1353876/0001127602-17-009249-2
1353876/0001127602-17-009249-3
1353876/0001127602-17-009249-4
1353876/0001127602-17-009249-5
1353876/0001127602-17-009249-6
1353876/0001127602-17-009249-7
1353876/0001127602-17-009249-1-h
1353876/0001127602-17-009538-1
1353876/0001127602-17-012395-1-h
1353876/0001127602-17-012395-2-h
1353876/0001127602-17-012395-1
1353876/0001127602-17-012395-2
1353876/0001127602-17-012395-1-h
1635175/0000899243-17-006396-1
1635175/0000899243-17-006396-1
1293332/0000707549-17-000002-1
1293332/0000707549-17-000002-2
1293332/0000707549-17-000015-1
1293332/0000707549-17-000022-1
1293332/0000707549-17-000022-1
1293332/0000707549-17-000

315293/0001179110-17-002848-1
315293/0001179110-17-002848-2
315293/0001179110-17-002848-3
315293/0001179110-17-002848-1
315293/0001179110-17-002848-2
315293/0001179110-17-002848-3
315293/0001179110-17-002848-4
315293/0001179110-17-002848-5
315293/0001179110-17-002848-6
315293/0001179110-17-002849-1
315293/0001179110-17-002849-2
315293/0001179110-17-002849-3
315293/0001179110-17-002849-1
315293/0001179110-17-002849-2
315293/0001179110-17-002849-3
315293/0001179110-17-002849-4
315293/0001179110-17-002849-5
315293/0001179110-17-002849-6
315293/0001179110-17-002850-1
315293/0001179110-17-002850-2
315293/0001179110-17-002850-3
315293/0001179110-17-002850-4
315293/0001179110-17-002850-1
315293/0001179110-17-002850-2
315293/0001179110-17-002850-3
315293/0001179110-17-002850-4
315293/0001179110-17-002850-5
315293/0001179110-17-002850-6
315293/0001179110-17-002850-7
315293/0001179110-17-002850-8
315293/0001179110-17-002851-1
315293/0001179110-17-002851-2
315293/0001179110-17-002851-3
315293/000

1671584/0001019056-17-000113-1
1671584/0001019056-17-000113-1
1671584/0001019056-17-000232-1
1671584/0001019056-17-000234-1
1671584/0001019056-17-000236-1
1671584/0001019056-17-000238-1
1671584/0001019056-17-000240-1
1671584/0001019056-17-000286-1
1671584/0001019056-17-000286-2
1671584/0001019056-17-000286-3
1671584/0001019056-17-000286-1
1671584/0001019056-17-000286-2
1671584/0001019056-17-000286-3
1671584/0001019056-17-000286-4
1671584/0001019056-17-000286-5
1671584/0001019056-17-000286-6
1671584/0001019056-17-000288-1
1671584/0001019056-17-000288-2
1671584/0001019056-17-000288-3
1671584/0001019056-17-000288-1
1671584/0001019056-17-000288-2
1671584/0001019056-17-000288-3
1671584/0001019056-17-000288-4
1671584/0001019056-17-000288-5
1671584/0001019056-17-000288-6
1671584/0001019056-17-000290-1
1671584/0001019056-17-000290-2
1671584/0001019056-17-000290-3
1671584/0001019056-17-000290-1
1671584/0001019056-17-000290-2
1671584/0001019056-17-000290-3
1671584/0001019056-17-000290-4
1671584/

4281/0000004281-17-000015-1
4281/0000004281-17-000015-1
4281/0000004281-17-000015-2
4281/0000004281-17-000015-3
4281/0000004281-17-000015-1-h
4281/0000004281-17-000016-1
4281/0000004281-17-000016-1
4281/0000004281-17-000017-1
4281/0000004281-17-000017-1
4281/0000004281-17-000017-2
4281/0000004281-17-000017-3
4281/0000004281-17-000017-1-h
4281/0000004281-17-000018-1
4281/0000004281-17-000018-1
4281/0000004281-17-000018-2
4281/0000004281-17-000018-3
4281/0000004281-17-000018-1-h
4281/0000004281-17-000019-1
4281/0000004281-17-000019-1-h
4281/0000004281-17-000020-1
4281/0000004281-17-000020-2
4281/0000004281-17-000021-1
4281/0000004281-17-000022-1
4281/0000004281-17-000022-1
4281/0000004281-17-000023-1
4281/0000004281-17-000023-2
4281/0000004281-17-000023-3
4281/0000004281-17-000023-1-h
4281/0000004281-17-000024-1
4281/0000004281-17-000024-2
4281/0000004281-17-000024-3
4281/0000004281-17-000024-4
4281/0000004281-17-000024-1-h
4281/0000004281-17-000025-1
4281/0000004281-17-000025-2
4281/000

In [26]:
conn = connectToDb(database)
createTable(conn, 'form4Head', headTblFields)
createTable(conn, 'form4dT', dTTblSql)
createTable(conn, 'form4nDT', nDTTblSql)
createTable(conn, 'form4footNote', footNoteTblFields)

#fname = "2007_4A_accNum" #fname = "flowers_com_inc2014.txt"
fname = "2017Form4.csv"
#fname = "2017Form4a" #400 4/a

with open(fname) as f:
    urls = f.read().splitlines()

#thisTime = urls #400 forms 4as
thisTime = urls[8001:9001] #5000 form 4s
#thisTime = urls[4001:4002]
for link in thisTime:
    links = link.split(",")
    url = "https://www.sec.gov/Archives/" + links[len(links)-1] #for 4s
    #url = "https://www.sec.gov/Archives/edgar/data/1305168/0000899243-17-004732.txt"
    #url = "https://www.sec.gov/Archives/" + link + ".txt" #for 2014form4a    
    try:
        response = request.urlopen(url)
    except error.HTTPError as err:
        failed.append(url)
        continue
    
    the_page = response.read()
    content = the_page.decode(encoding='latin-1')
    file = open("test", "w")
    file.write(content) 

    #parts = link.split("/")
    parts = links[len(links)-1].split('/')
    if len(parts) == 5: #.../data/1084869/000108486914000025/0001084869-14-000025.txt
        accNum = parts[len(parts)-3] + '/' +parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]
    if len(parts) == 4: #.../data/1214101/0001104659-07-084171.txt
        accNum = parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]
        
    # start parsing
    begin = content.find("<SEC-DOCUMENT>")
    end = content.find("-----END")
    xmlFile = content[begin:end]
    soup = BeautifulSoup(xmlFile, 'xml')
    parseHead(soup, accNum)
    parseTransacs(soup, accNum)


{'rptOwnerBusinessPhone': '2125062999', 'documentType': '4', 'issuerIncorpState': 'PA', 'issuerBusinessStreet1': '390 PARK AVENUE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'ARNC', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '10022-4608', 'issuerBusinessZip': '10022-4608', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'NEW YORK', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10019', 'rptOwnerSecFileNum': '001-03610', 'headId': '4281/0001140361-17-005168-1', 'filedDate': '20170208', 'issuerBusinessState': 'NY', 'rptOwnerMailZip': '10019', 'rptOwnerState': 'NY', 'issuerCik': '0000004281', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '40 WEST 57TH STREET', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '40 WEST 57TH STREET, 30TH FLOOR', 'issuerIndustrialClassification': 'ROLLING DRAWING  EXTRUDING OF NONFERROUS METALS [3350]', 'rptOwnerisOther': '0', 'rptOwnerMailState': '

{'rptOwnerBusinessPhone': '2125062999', 'documentType': '4', 'issuerIncorpState': 'PA', 'issuerBusinessStreet1': '390 PARK AVENUE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'ARNC', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '10022-4608', 'issuerBusinessZip': '10022-4608', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'NEW YORK', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10019', 'rptOwnerSecFileNum': '001-03610', 'headId': '4281/0001140361-17-006400-1', 'filedDate': '20170214', 'issuerBusinessState': 'NY', 'rptOwnerMailZip': '10019', 'rptOwnerState': 'NY', 'issuerCik': '0000004281', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '40 WEST 57TH STREET', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '40 WEST 57TH STREET, 30TH FLOOR', 'issuerIndustrialClassification': 'ROLLING DRAWING  EXTRUDING OF NONFERROUS METALS [3350]', 'rptOwnerisOther': '0', 'rptOwnerMailState': '

{'rptOwnerBusinessPhone': '2125062999', 'documentType': '4', 'issuerIncorpState': 'PA', 'issuerBusinessStreet1': '390 PARK AVENUE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'ARNC', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '10022-4608', 'issuerBusinessZip': '10022-4608', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'NEW YORK', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10019', 'rptOwnerSecFileNum': '001-03610', 'headId': '4281/0001140361-17-007996-1', 'filedDate': '20170216', 'issuerBusinessState': 'NY', 'rptOwnerMailZip': '10019', 'rptOwnerState': 'NY', 'issuerCik': '0000004281', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '40 WEST 57TH STREET', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '40 WEST 57TH STREET, 30TH FLOOR', 'issuerIndustrialClassification': 'ROLLING DRAWING  EXTRUDING OF NONFERROUS METALS [3350]', 'rptOwnerisOther': '0', 'rptOwnerMailState': '

{'rptOwnerBusinessPhone': '2125062999', 'documentType': '4', 'issuerIncorpState': 'PA', 'issuerBusinessStreet1': '390 PARK AVENUE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'ARNC', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '10022-4608', 'issuerBusinessZip': '10022-4608', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'NEW YORK', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10019', 'rptOwnerSecFileNum': '001-03610', 'headId': '4281/0001140361-17-008698-1', 'filedDate': '20170222', 'issuerBusinessState': 'NY', 'rptOwnerMailZip': '10019', 'rptOwnerState': 'NY', 'issuerCik': '0000004281', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '40 WEST 57TH STREET', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '40 WEST 57TH STREET, 30TH FLOOR', 'issuerIndustrialClassification': 'ROLLING DRAWING  EXTRUDING OF NONFERROUS METALS [3350]', 'rptOwnerisOther': '0', 'rptOwnerMailState': '

{'documentType': '4', 'issuerIncorpState': 'PA', 'issuerBusinessStreet1': '390 PARK AVENUE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'ARNC', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '10022-4608', 'issuerBusinessZip': '10022-4608', 'rptOwnerisTenPercentOwner': '0', 'issuerBusinessCity': 'NEW YORK', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10022', 'rptOwnerSecFileNum': '001-03610', 'headId': '4281/0000004281-17-000044-1', 'filedDate': '20170314', 'issuerBusinessState': 'NY', 'rptOwnerMailZip': '10022', 'rptOwnerState': 'NY', 'issuerCik': '0000004281', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '390 PARK AVENUE', 'rptOwnerStreet2': '', 'notSubjectToSection16': None, 'schemaVersion': 'X0306', 'rptOwnerStreet1': '390 PARK AVENUE', 'issuerIndustrialClassification': 'ROLLING DRAWING  EXTRUDING OF NONFERROUS METALS [3350]', 'rptOwnerisOther': '0', 'rptOwnerMailState': 'NY', 'rptOwnerCity': 'NEW YORK,', 'rowNumber': 1, 'rptOwne

...

{'documentType': '4', 'issuerIncorpState': 'OH', 'issuerBusinessStreet1': '138 PUTNAM ST', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'PEBO', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'MARIETTA', 'issuerMailZip': '45750-0738', 'issuerBusinessZip': '45750-0738', 'rptOwnerisTenPercentOwner': '0', 'issuerBusinessCity': 'MARIETTA', 'issuerFiscalYrEnd': '0202', 'rptOwnerZipCode': '45750', 'rptOwnerSecFileNum': '000-16772', 'headId': '1105134/0000318300-17-000051-1', 'filedDate': '20170202', 'issuerBusinessState': 'OH', 'rptOwnerMailZip': '45750', 'rptOwnerState': 'OH', 'changedDate': '20170202', 'issuerCik': '0000318300', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '138 PUTNAM STREET', 'rptOwnerStreet2': 'P.O. BOX 738', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '138 PUTNAM STREET', 'issuerIndustrialClassification': 'NATIONAL COMMERCIAL BANKS [6021]', 'rptOwnerisOther': '0', 'rptOwnerMailState': 'OH', 'rptOwnerCity': 'MARIETTA', 'row

{'documentType': '4', 'issuerIncorpState': 'DE', 'issuerBusinessStreet1': '10220 SOUTH 51ST STREET', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'INSY', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'PHOENIX', 'issuerMailZip': '85044', 'issuerBusinessZip': '85044', 'rptOwnerisTenPercentOwner': '0', 'issuerBusinessCity': 'PHOENIX', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '85286', 'rptOwnerSecFileNum': '001-35902', 'headId': '1044831/0001209191-17-014090-1', 'filedDate': '20170223', 'issuerBusinessState': 'AZ', 'rptOwnerMailZip': '85034', 'rptOwnerState': 'AZ', 'changedDate': '20170223', 'issuerCik': '0001516479', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '1520 EAST PIMA ST', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '1333 SOUTH SPECTRUM BLVD, SUITE 100', 'issuerIndustrialClassification': 'PHARMACEUTICAL PREPARATIONS [2834]', 'rptOwnerisOther': '0', 'rptOwnerMailState': 'AZ', 'rptOwnerCity': 'CHANDLER'

{'rptOwnerBusinessPhone': '212-339-5633', 'documentType': '4', 'issuerIncorpState': 'DE', 'issuerBusinessStreet1': '1801 AUGUSTINE CUT-OFF', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'INCY', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '19803', 'issuerBusinessZip': '19803', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'WILMINGTON', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10065', 'rptOwnerSecFileNum': '001-12400', 'headId': '1087940/0001638599-17-000024-1', 'filedDate': '20170104', 'issuerBusinessState': 'DE', 'rptOwnerMailZip': '10065', 'rptOwnerState': 'NY', 'issuerCik': '0000879169', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '667 MADISON AVENUE', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '667 MADISON AVENUE, 21ST FLOOR', 'issuerIndustrialClassification': 'SERVICES-COMMERCIAL PHYSICAL  BIOLOGICAL RESEARCH [8731]', 'rptOwnerisOther': '0', 'rptOwnerMailState

{'rptOwnerBusinessPhone': '212-339-5633', 'documentType': '4', 'issuerBusinessStreet1': '301 PENOBSCOT DRIVE', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'GHDX', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '94063', 'issuerBusinessZip': '94063', 'rptOwnerisTenPercentOwner': '1', 'issuerBusinessCity': 'REDWOOD CITY', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10065', 'rptOwnerSecFileNum': '000-51541', 'headId': '1087940/0001638599-17-000029-1', 'filedDate': '20170104', 'issuerBusinessState': 'CA', 'rptOwnerMailZip': '10065', 'rptOwnerState': 'NY', 'issuerCik': '0001131324', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '667 MADISON AVENUE', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '667 MADISON AVENUE, 21ST FLOOR', 'issuerIndustrialClassification': 'SERVICES-MEDICAL LABORATORIES [8071]', 'rptOwnerisOther': '0', 'rptOwnerMailState': 'NY', 'rptOwnerCity': 'NEW YORK', 'rowNumber'

1087940/0001638599-17-000029-1-h
{'rptOwnerBusinessPhone': '212-339-5600', 'documentType': '4', 'issuerIncorpState': 'DE', 'issuerBusinessStreet1': '167 SIDNEY STREET', 'rptOwnerSecAct': '1934 Act', 'issuerTradingSymbol': 'IDRA', 'rptOwnerFormType': '4', 'rptOwnerMailCity': 'NEW YORK', 'issuerMailZip': '02139', 'issuerBusinessZip': '02139', 'rptOwnerisTenPercentOwner': '0', 'issuerBusinessCity': 'CAMBRIDGE', 'issuerFiscalYrEnd': '1231', 'rptOwnerZipCode': '10065', 'rptOwnerSecFileNum': '001-31918', 'headId': '1087940/0001638599-17-000048-1', 'filedDate': '20170105', 'issuerBusinessState': 'MA', 'rptOwnerMailZip': '10065', 'rptOwnerState': 'NY', 'issuerCik': '0000861838', 'dateOfOriginalSubmission': None, 'rptOwnerMailStreet1': '667 MADISON AVE.', 'rptOwnerStreet2': '', 'notSubjectToSection16': '0', 'schemaVersion': 'X0306', 'rptOwnerStreet1': '667 MADISON AVENUE, 21ST FLOOR', 'issuerIndustrialClassification': 'BIOLOGICAL PRODUCTS (NO DIAGNOSTIC SUBSTANCES) [2836]', 'rptOwnerisOther': '

In [12]:
dropHeadFields = ['accNum', 'rowNumber', 'documentType', 'filedDate', 'changedDate', 'periodOfReport', 'dateOfOriginalSubmission', "headId", 'rptOwnerFormType', 'rptOwnerSecAct', 'rptOwnerSecFileNum', 'rptOwnerFilmNum', 'rptOwnerBusinessPhone', 'rptOwnerStateDescription']

def filterFields(allFields, dropFields):
    fields = allFields.replace("\n", "").split(",")
    fields = [field.split(" ")[0] for field in fields]
    fields = [field for field in fields if field not in dropFields and field != '']
    return fields
headFields = filterFields(headTblFields, dropHeadFields)
headFields

['publicDocCount',
 'schemaVersion',
 'notSubjectToSection16',
 'issuerName',
 'issuerCik',
 'issuerIndustrialClassification',
 'issuerIrs',
 'issuerIncorpState',
 'issuerFiscalYrEnd',
 'issuerBusinessStreet1',
 'issuerBusinessCity',
 'issuerBusinessState',
 'issuerBusinessZip',
 'issuerBusinessPhone',
 'issuerMailStreet1',
 'issuerMailStreet2',
 'issuerMailCity',
 'issuerMailState',
 'issuerMailZip',
 'issuerTradingSymbol',
 'rptOwnerName',
 'rptOwnerCik',
 'rptOwnerStreet1',
 'rptOwnerStreet2',
 'rptOwnerCity',
 'rptOwnerState',
 'rptOwnerZipCode',
 'rptOwnerisDirector',
 'rptOwnerisOfficer',
 'rptOwnerisTenPercentOwner',
 'rptOwnerisOther']

In [7]:
query = "select A.accNum as aAcc, B.accNum as bAcc from form4head A, form4head B where A.documentType = '4/A' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate"
for field in headFields:
    query += " and A." + field + " = " "B." + field
query += ";"
print(query)

cur = conn.cursor()
cur.execute(query)
matches = cur.fetchall()
matches

select A.accNum as aAcc, B.accNum as bAcc from form4head A, form4head B where A.documentType = '4/A' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate and A.publicDocCount = B.publicDocCount and A.schemaVersion = B.schemaVersion and A.notSubjectToSection16 = B.notSubjectToSection16 and A.issuerName = B.issuerName and A.issuerCik = B.issuerCik and A.issuerIndustrialClassification = B.issuerIndustrialClassification and A.issuerIrs = B.issuerIrs and A.issuerIncorpState = B.issuerIncorpState and A.issuerFiscalYrEnd = B.issuerFiscalYrEnd and A.issuerBusinessStreet1 = B.issuerBusinessStreet1 and A.issuerBusinessCity = B.issuerBusinessCity and A.issuerBusinessState = B.issuerBusinessState and A.issuerBusinessZip = B.issuerBusinessZip and A.issuerBusinessPhone = B.issuerBusinessPhone and A.issuerMailStreet1 = B.issuerMailStreet1 and A.issuerMailStreet2 = B.issuerMailStreet2 and A.issuerMailCity = B.issuerMailCity and A.issuerMailState = B.issuerMailState and A.issuerMailZip

NameError: name 'conn' is not defined

In [13]:
dropNDTFields = ['accNum', 'nDTId', 'rowNumber', 'footNoteId']
ndtFields = filterFields(nDTTblFields, dropNDTFields)
dropDTFields = ['accNum', 'dTId', 'rowNumber', 'footNoteId']
dtFields = filterFields(dTTblFields, dropDTFields)

ndtFields

['securityTitle',
 'transactionDate',
 'transactionFormType',
 'transactionCode',
 'equitySwapInvolved',
 'transactionTimelines',
 'transactionShares',
 'transactionPricePerShare',
 'transactionAcquiredDisposedCode',
 'sharesOwnedFollowingTransaction',
 'directOrIndirectOwnership',
 'natureOfOwnership']

In [17]:
query = "select"
for field in ndtFields:
    query += " A." + field + " as a_" + field + ", B." + field + " as b_" + field +","
query = query[:len(query)-1] + " from form4ndT A, form4ndT B where A.accNum = '1084869/000108486914000025/0001084869-14-000025' and B.accNum = '1084869/000108486914000024/0001084869-14-000024';";
print(query)
#maybe dont do this, just compare fields from 2 transacs one by one....

select A.securityTitle as a_securityTitle, B.securityTitle as b_securityTitle, A.transactionDate as a_transactionDate, B.transactionDate as b_transactionDate, A.transactionFormType as a_transactionFormType, B.transactionFormType as b_transactionFormType, A.transactionCode as a_transactionCode, B.transactionCode as b_transactionCode, A.equitySwapInvolved as a_equitySwapInvolved, B.equitySwapInvolved as b_equitySwapInvolved, A.transactionTimelines as a_transactionTimelines, B.transactionTimelines as b_transactionTimelines, A.transactionShares as a_transactionShares, B.transactionShares as b_transactionShares, A.transactionPricePerShare as a_transactionPricePerShare, B.transactionPricePerShare as b_transactionPricePerShare, A.transactionAcquiredDisposedCode as a_transactionAcquiredDisposedCode, B.transactionAcquiredDisposedCode as b_transactionAcquiredDisposedCode, A.sharesOwnedFollowingTransaction as a_sharesOwnedFollowingTransaction, B.sharesOwnedFollowingTransaction as b_sharesOwnedFol

In [28]:
conn = connectToDb(database)
cur = conn.cursor()

In [39]:
#for match in matches:
# aDT = cur.execute("select * from form4dT where accNum = '" + match[0]+ + "';").fetchall()
# bDT = cur.execute("select * from form4dT where accNum = '" + match[0]+ + "';").fetchall()


match= ["1084869/000108486914000025/0001084869-14-000025", "1084869/000108486914000024/0001084869-14-000024"]


aNDT = cur.execute("select nDTId," + ','.join(ndtFields) +" from form4ndT where accNum = '" + match[0]+ "';").fetchall()
oNDT = cur.execute("select nDTId," + ','.join(ndtFields) +" from form4ndT where accNum = '" + match[1]+ "';").fetchall()

'''
# max matches; dont account for new records
matchingTransacs = {}
for each transaction in amendment:
    numMatches = 0
    originalTransac = None
    for each transaction in original:
        thisMatches = num(fields that match)
        if thisMatches > numMatches:
            numMatches = thisMatches
            originalTransac = transaction
'''
# matchingTransacs = {}
# for a in aNDT:
#     mostMatches = 0
#     originalTransac = None
#     for o in oNDT:
#         thisMatches = sum([1 for i in range(len(ndtFields)) if a[i] == o[i]])
#         if thisMatches > mostMatches:
#             originalTransac = o
#     matchingTransacs[a[0]] = originalTransac[0]
#     oNDT.remove(originalTransac)
# matchingTransacs



{'1084869/000108486914000025/0001084869-14-000025-1': '1084869/000108486914000024/0001084869-14-000024-1'}

In [36]:
a

('1084869/000108486914000025/0001084869-14-000025-1',
 '<value>Class A Common Stock</value>',
 '<value>2014-12-08</value>',
 '4',
 'A',
 '0',
 '0',
 '<value>2516</value>',
 '<value>0</value><footnoteId id="F1"/>',
 '<value>A</value>',
 '<value>30808</value>',
 '<value>D</value>',
 None)

In [13]:
def main():
    conn = connectToDb(database)
    createTable(conn, 'form4Head', headTblFields)
    createTable(conn, 'form4dT', dTTblSql)
    createTable(conn, 'form4nDT', nDTTblSql)
    createTable(conn, 'form4footNote', footNoteTblFields)
    
    fname = "2007_4A_accNum"
    with open(fname) as f:
        urls = f.read().splitlines()
        
    i = 0
    for link in urls:
        url = "https://www.sec.gov/Archives/" + link
        response = urllib.request.urlopen(url)
        the_page = response.read()
        content = the_page.decode(encoding='latin-1')
        file = open("test", "w")
        file.write(content) 

        parts = urls[1].split('/')
        accNum = parts[len(parts)-1].split('.')[0]
        #accNum = '0001104659-07-084171'

        # start parsing
        begin = content.find("<SEC-DOCUMENT>")
        end = content.find("</SEC-DOCUMENT>")#("-----END")
        xmlFile = content[begin:end]
        soup = BeautifulSoup(xmlFile, 'xml')
        
        parseHead(soup, accNum)
        parseTransacs(soup, accNum)
        i+=1
        if i ==50:
            break

In [14]:
main()

NameError: name 'soup' is not defined

In [106]:
fname = "test.txt"
with open(fname) as f:
    urls = f.read()
soup = BeautifulSoup(urls, 'html')
soup

<?xml version="1.0" encoding="ISO-8859-1" ?><html><body><feed xmlns="http://www.w3.org/2005/Atom">
<title>Archive of Historical EDGAR Documents</title>
<link href="http://www.sec.gov//cgi-bin/srch-edgar?text=TYPE%3D4%2FA&amp;start=321&amp;count=80&amp;first=2017&amp;last=2017&amp;output=atom" rel="alternate"/>
<link href="http://www.sec.gov//cgi-bin/srch-edgar?text=TYPE%3D4%2FA&amp;start=321&amp;count=80&amp;first=2017&amp;last=2017&amp;output=atom" rel="self"/>
<id>http://www.sec.gov//cgi-bin/srch-edgar?text=TYPE%3D4%2FA&amp;start=321&amp;count=80&amp;first=2017&amp;last=2017&amp;output=atom</id>
<author><name>Webmaster</name><email>webmaster@sec.gov</email></author>
<updated>2018-10-08T20:26:36-04:00</updated>
<entry>
<title>4/A - Andersons, Inc.</title>
<link href="/Archives/edgar/data/821026/000120919117019159/0001209191-17-019159-index.htm" rel="alternate" type="text/html"/>
<summary type="html">&lt;b&gt;Filed Date:&lt;/b&gt; 03/08/2017 &lt;b&gt;Accession Number:&lt;/b&gt; 0001209

In [107]:
import re
a = soup.find_all("link", href=re.compile("Archives"))
#links = [i.get("href").replace("-index.htm", "").replace("/Archives/", "") for i in a]

file = open("2017Form4a", "a+")
for i in a:
    link = i.get("href").replace("-index.htm", "").replace("/Archives/", "")
    file.write(link + "\n") 

In [136]:
with open("2017Form4a") as f:
    urls = f.read().splitlines()
print(len(urls))
urls = set(urls)
print(len(urls))
file = open("2017Form4a", "w")
for i in urls:
    file.write(i + "\n") 

477
401


0
6
10
